# Customer.IO Production Deployment

Comprehensive production deployment and infrastructure management for Customer.IO data pipelines including:
- Container orchestration and deployment strategies
- Configuration management and secrets handling
- Infrastructure as Code (IaC) templates
- CI/CD pipeline integration
- Environment management and promotion
- Performance optimization and scaling
- Security hardening and compliance
- Disaster recovery and backup strategies

In [ ]:
# Essential imports for production deployment
import json
import yaml
import os
import shutil
import subprocess
import tempfile
from datetime import datetime, timezone, timedelta
from typing import Dict, List, Optional, Any, Union, Set, Tuple
from dataclasses import dataclass, field
from pathlib import Path
from enum import Enum
import structlog
from pydantic import BaseModel, Field, validator
import hashlib
import base64
import secrets
import warnings

# Suppress warnings for cleaner output
warnings.filterwarnings('ignore')

print("SUCCESS: Core deployment imports loaded")

In [ ]:
# Import Customer.IO utilities with existing DeploymentManager
import sys
import os

# Import using proper package structure (no need to modify sys.path)
from utils.api_client import CustomerIOClient
from utils.deployment_manager import (
    DeploymentManager,
    Environment,
    DeploymentStrategy,
    InfrastructureProvider,
    DeploymentStatus,
    SecurityLevel,
    SecretConfig,
    EnvironmentConfig,
    DeploymentConfig,
    DeploymentResult,
    InfrastructureTemplate,
    ConfigurationManager,
    InfrastructureGenerator,
    DeploymentEngine
)
from utils.error_handlers import retry_on_error, ErrorContext, CustomerIOError
from utils.validators import validate_request_size

print("SUCCESS: Customer.IO utilities and DeploymentManager imported")

## Using Existing DeploymentManager

The deployment components are already implemented in `utils/deployment_manager.py`. This notebook demonstrates usage of the existing manager.

In [ ]:
# Core enums are imported from utils/deployment_manager.py
print("Available environments:", [env.value for env in Environment])
print("Available deployment strategies:", [strategy.value for strategy in DeploymentStrategy])
print("Available infrastructure providers:", [provider.value for provider in InfrastructureProvider])
print("Available deployment statuses:", [status.value for status in DeploymentStatus])
print("Available security levels:", [level.value for level in SecurityLevel])

print("SUCCESS: Using existing deployment enums from DeploymentManager")

In [ ]:
# Example: Creating deployment configurations using existing models
try:
    # Example secret configuration
    sample_secret = SecretConfig(
        name="CUSTOMER_IO_API_KEY",
        source="env",
        encrypted=True,
        required=True,
        description="Customer.IO API key for authentication"
    )
    
    print("Created sample secret config:")
    print(f"  Name: {sample_secret.name}")
    print(f"  Source: {sample_secret.source}")
    print(f"  Required: {sample_secret.required}")
    print(f"  Encrypted: {sample_secret.encrypted}")
    
    # Example environment configuration
    sample_env = EnvironmentConfig(
        name=Environment.DEVELOPMENT,
        display_name="Development Environment",
        description="Local development environment",
        provider=InfrastructureProvider.DOCKER,
        region="us-east-1",
        replicas=1,
        resources={
            "requests": {"memory": "256Mi", "cpu": "250m"},
            "limits": {"memory": "512Mi", "cpu": "500m"}
        },
        secrets=[sample_secret],
        config_vars={"LOG_LEVEL": "DEBUG", "DEBUG": "true"},
        security_level=SecurityLevel.BASIC,
        auto_scaling=False,
        monitoring_enabled=True
    )
    
    print(f"\nCreated sample environment config:")
    print(f"  Name: {sample_env.name}")
    print(f"  Provider: {sample_env.provider}")
    print(f"  Replicas: {sample_env.replicas}")
    print(f"  Security Level: {sample_env.security_level}")
    print(f"  Auto-scaling: {sample_env.auto_scaling}")
    
    # Example deployment configuration
    sample_deployment = DeploymentConfig(
        deployment_id="test-deployment-123",
        name="customer-io-pipelines",
        version="2.0.0",
        environment=Environment.DEVELOPMENT,
        strategy=DeploymentStrategy.ROLLING_UPDATE,
        image_tag="2.0.0",
        created_by="developer@company.com"
    )
    
    print(f"\nCreated sample deployment config:")
    print(f"  ID: {sample_deployment.deployment_id}")
    print(f"  Version: {sample_deployment.version}")
    print(f"  Strategy: {sample_deployment.strategy}")
    print(f"  Created by: {sample_deployment.created_by}")
    print(f"  Created at: {sample_deployment.created_at}")
    
    print("SUCCESS: Using existing deployment configuration models")
    
except Exception as e:
    print(f"ERROR: Failed to create configurations: {e}")

In [ ]:
# Example: Creating deployment results and infrastructure templates  
try:
    # Example deployment result
    deployment_result = DeploymentResult(
        deployment_id="test-deployment-123",
        status=DeploymentStatus.RUNNING,
        start_time=datetime.now(timezone.utc)
    )
    
    # Add some logs
    deployment_result.add_log("Deployment started")
    deployment_result.add_log("Validating configuration")
    deployment_result.add_log("Rolling out to instances")
    
    # Add performance metrics
    deployment_result.performance_metrics = {
        "step_1_duration_ms": 150.0,
        "step_2_duration_ms": 200.0,
        "health_check_success_rate": 100.0
    }
    
    print("Created deployment result:")
    print(f"  ID: {deployment_result.deployment_id}")
    print(f"  Status: {deployment_result.status}")
    print(f"  Logs: {len(deployment_result.logs)} entries")
    print(f"  Metrics: {len(deployment_result.performance_metrics)} items")
    print(f"  Is successful: {deployment_result.is_successful()}")
    
    # Simulate finishing deployment
    deployment_result.finish(DeploymentStatus.SUCCESS)
    print(f"  Final duration: {deployment_result.duration_seconds:.2f} seconds")
    
    # Example infrastructure template
    sample_template = InfrastructureTemplate(
        template_id="k8s-dev-2.0.0",
        name="Kubernetes Development Deployment",
        provider=InfrastructureProvider.KUBERNETES,
        template_format="yaml",
        template_content="apiVersion: apps/v1\nkind: Deployment\n...",
        parameters={
            "environment": "development",
            "version": "2.0.0",
            "replicas": 1
        },
        description="Kubernetes deployment template for development environment"
    )
    
    print(f"\nCreated infrastructure template:")
    print(f"  ID: {sample_template.template_id}")
    print(f"  Name: {sample_template.name}")
    print(f"  Provider: {sample_template.provider}")
    print(f"  Format: {sample_template.template_format}")
    print(f"  Parameters: {sample_template.parameters}")
    print(f"  Content size: {len(sample_template.template_content)} characters")
    
    print("SUCCESS: Using existing deployment result and template models")
    
except Exception as e:
    print(f"ERROR: Failed to create models: {e}")

## Using Existing ConfigurationManager

The ConfigurationManager is already implemented in `utils/deployment_manager.py` with secure secrets management.

In [ ]:
# Example: Using existing ConfigurationManager
try:
    # Initialize ConfigurationManager
    config_manager = ConfigurationManager()
    
    # Create environment configuration
    dev_env = EnvironmentConfig(
        name=Environment.DEVELOPMENT,
        display_name="Development",
        description="Development environment for testing",
        provider=InfrastructureProvider.DOCKER,
        region="us-east-1",
        replicas=1,
        resources={
            "requests": {"memory": "256Mi", "cpu": "250m"},
            "limits": {"memory": "512Mi", "cpu": "500m"}
        },
        secrets=[
            SecretConfig(name="API_KEY", source="env", description="API key"),
            SecretConfig(name="DB_PASSWORD", source="vault", description="Database password")
        ],
        config_vars={"LOG_LEVEL": "DEBUG", "ENVIRONMENT": "development"},
        security_level=SecurityLevel.BASIC,
        auto_scaling=False,
        monitoring_enabled=True
    )
    
    # Register environment
    config_manager.register_environment(dev_env)
    
    # Set some secrets
    config_manager.set_secret(Environment.DEVELOPMENT, "API_KEY", "test-key-123", encrypt=True)
    config_manager.set_secret(Environment.DEVELOPMENT, "DB_PASSWORD", "secure-password", encrypt=True)
    
    print("ConfigurationManager Usage:")
    print(f"  Environment registered: {dev_env.name}")
    print(f"  Secrets configured: {len(dev_env.secrets)}")
    print(f"  Config variables: {len(dev_env.config_vars)}")
    
    # Retrieve secret (decrypted)
    api_key = config_manager.get_secret(Environment.DEVELOPMENT, "API_KEY")
    print(f"  Retrieved API key: {api_key[:8]}..." if api_key else "  API key: Not found")
    
    # Generate configuration file
    config_yaml = config_manager.generate_config_file(Environment.DEVELOPMENT, "yaml")
    print(f"  Config file generated: {len(config_yaml)} characters")
    
    # Validate environment
    validation = config_manager.validate_environment(Environment.DEVELOPMENT)
    print(f"  Environment validation: {'Valid' if validation['valid'] else 'Invalid'}")
    if validation['warnings']:
        print(f"  Warnings: {len(validation['warnings'])}")
    
    # Get metrics
    metrics = config_manager.get_metrics()
    print(f"  Metrics: {metrics['environments_registered']} environments, {metrics['total_secrets']} secrets")
    
    print("SUCCESS: Using existing ConfigurationManager with encryption")
    
except Exception as e:
    print(f"ERROR: Failed to use ConfigurationManager: {e}")

## Using Existing InfrastructureGenerator

The InfrastructureGenerator is already implemented with support for multiple cloud providers and orchestration platforms.

# Example: Using existing InfrastructureGenerator
try:
    # Initialize InfrastructureGenerator (already implemented in utils/deployment_manager.py)
    infra_generator = InfrastructureGenerator()
    
    # Example environment configuration for testing
    test_env = EnvironmentConfig(
        name=Environment.DEVELOPMENT,
        display_name="Development Environment",
        description="Development environment for testing infrastructure generation",
        provider=InfrastructureProvider.KUBERNETES,
        region="us-east-1",
        namespace="customer-io-dev",
        replicas=2,
        resources={
            "requests": {"memory": "512Mi", "cpu": "500m"},
            "limits": {"memory": "1Gi", "cpu": "1000m"}
        },
        secrets=[
            SecretConfig(name="API_KEY", source="k8s", description="API key"),
            SecretConfig(name="DB_URL", source="k8s", description="Database URL")
        ],
        config_vars={"LOG_LEVEL": "DEBUG", "ENVIRONMENT": "development"},
        security_level=SecurityLevel.STANDARD,
        auto_scaling=True,
        monitoring_enabled=True
    )
    
    # Example deployment configuration
    test_deployment = DeploymentConfig(
        deployment_id="test-deployment-123",
        name="customer-io-pipelines",
        version="2.0.0",
        environment=Environment.DEVELOPMENT,
        strategy=DeploymentStrategy.ROLLING_UPDATE,
        image_tag="2.0.0",
        created_by="developer@company.com"
    )
    
    # Generate Kubernetes deployment template
    k8s_template = infra_generator.generate_kubernetes_deployment(test_env, test_deployment)
    
    print("Generated Kubernetes Template:")
    print(f"  Template ID: {k8s_template.template_id}")
    print(f"  Name: {k8s_template.name}")
    print(f"  Provider: {k8s_template.provider}")
    print(f"  Format: {k8s_template.template_format}")
    print(f"  Parameters: {k8s_template.parameters}")
    print(f"  Content size: {len(k8s_template.template_content)} characters")
    
    # Show template preview
    lines = k8s_template.template_content.split('\n')[:15]
    print(f"  Template preview:")
    for line in lines:
        print(f"    {line}")
    print(f"    ... ({len(k8s_template.template_content.split('\n')) - 15} more lines)")
    
    print("SUCCESS: Using existing InfrastructureGenerator from DeploymentManager")
    
except Exception as e:
    print(f"ERROR: Failed to use InfrastructureGenerator: {e}")

In [ ]:
## Using Existing DeploymentEngine

The DeploymentEngine is already implemented in `utils/deployment_manager.py` with strategy execution and rollback capabilities.

# Example: Using existing DeploymentEngine
try:
    # Initialize ConfigurationManager and DeploymentEngine
    config_manager = ConfigurationManager()
    deployment_engine = DeploymentEngine(config_manager)
    
    # Register a test environment
    test_env = EnvironmentConfig(
        name=Environment.DEVELOPMENT,
        display_name="Test Environment",
        description="Test environment for deployment engine",
        provider=InfrastructureProvider.DOCKER,
        region="us-east-1",
        replicas=1,
        resources={
            "requests": {"memory": "256Mi", "cpu": "250m"},
            "limits": {"memory": "512Mi", "cpu": "500m"}
        },
        secrets=[SecretConfig(name="API_KEY", source="env", description="API key")],
        config_vars={"LOG_LEVEL": "DEBUG"},
        security_level=SecurityLevel.BASIC,
        auto_scaling=False,
        monitoring_enabled=True
    )
    
    config_manager.register_environment(test_env)
    
    # Create deployment configuration
    deployment_config = DeploymentConfig(
        deployment_id="test-deployment-engine-123",
        name="customer-io-pipelines",
        version="2.0.0",
        environment=Environment.DEVELOPMENT,
        strategy=DeploymentStrategy.ROLLING_UPDATE,
        image_tag="2.0.0",
        created_by="test-user"
    )
    
    print("Testing DeploymentEngine:")
    print(f"  Deployment ID: {deployment_config.deployment_id}")
    print(f"  Strategy: {deployment_config.strategy}")
    print(f"  Environment: {deployment_config.environment}")
    
    # Execute deployment
    result = deployment_engine.deploy(deployment_config)
    
    print(f"\nDeployment Result:")
    print(f"  Status: {result.status}")
    print(f"  Duration: {result.duration_seconds:.2f} seconds")
    print(f"  Health Check Passed: {result.health_check_passed}")
    print(f"  Rollback Performed: {result.rollback_performed}")
    print(f"  Log Entries: {len(result.logs)}")
    
    # Show first few log entries
    print(f"  Recent logs:")
    for log_entry in result.logs[-5:]:
        print(f"    {log_entry['timestamp']}: {log_entry['message']}")
    
    # Show performance metrics
    if result.performance_metrics:
        print(f"  Performance metrics:")
        for metric, value in result.performance_metrics.items():
            print(f"    {metric}: {value}")
    
    # Get deployment statistics
    stats = deployment_engine.get_deployment_statistics()
    print(f"\nDeployment Engine Statistics:")
    print(f"  Total deployments: {stats['total_deployments']}")
    print(f"  Success rate: {stats['success_rate_percent']:.1f}%")
    print(f"  Average deployment time: {stats['average_deployment_time_seconds']:.2f}s")
    print(f"  Active deployments: {stats['active_deployments']}")
    
    print("SUCCESS: Using existing DeploymentEngine with strategy execution")
    
except Exception as e:
    print(f"ERROR: Failed to use DeploymentEngine: {e}")

## Using Existing DeploymentManager

The complete DeploymentManager is already implemented in `utils/deployment_manager.py` combining all deployment components.

In [ ]:
# Initialize deployment manager
try:
    client = CustomerIOClient(
        api_key="test_key_456",
        region="us"
    )
    
    deployment_manager = DeploymentManager(client)
    
    print("SUCCESS: DeploymentManager initialized with default environments")
    
except Exception as e:
    print(f"ERROR: Failed to initialize DeploymentManager: {e}")

In [ ]:
# Example: Create and execute deployment
try:
    # Create deployment configuration
    deployment_config = deployment_manager.create_deployment(
        name="customer-io-pipelines",
        version="2.1.0",
        environment=Environment.DEVELOPMENT,
        strategy=DeploymentStrategy.ROLLING_UPDATE,
        created_by="developer@company.com"
    )
    
    print(f"Deployment Configuration Created:")
    print(f"  ID: {deployment_config.deployment_id}")
    print(f"  Environment: {deployment_config.environment}")
    print(f"  Strategy: {deployment_config.strategy}")
    print(f"  Version: {deployment_config.version}")
    
    # Execute deployment
    result = deployment_manager.deploy_application(deployment_config)
    
    print(f"\nDeployment Result:")
    print(f"  Status: {result.status}")
    print(f"  Duration: {result.duration_seconds:.2f} seconds")
    print(f"  Health Checks: {'Passed' if result.health_check_passed else 'Failed'}")
    print(f"  Rollback Performed: {result.rollback_performed}")
    
    if result.performance_metrics:
        print(f"  Performance Metrics:")
        for metric, value in result.performance_metrics.items():
            print(f"    {metric}: {value}")
    
except Exception as e:
    print(f"ERROR: Deployment failed: {e}")

In [ ]:
# Example: Generate infrastructure templates
try:
    # Generate templates for different environments
    environments_to_test = [Environment.DEVELOPMENT, Environment.STAGING, Environment.PRODUCTION]
    
    for env in environments_to_test:
        deployment_config = deployment_manager.create_deployment(
            name="customer-io-pipelines",
            version="2.1.0",
            environment=env
        )
        
        templates = deployment_manager.generate_infrastructure(env, deployment_config)
        
        print(f"\nInfrastructure Templates for {env}:")
        for template in templates:
            print(f"  - {template.name} ({template.provider})")
            print(f"    Template ID: {template.template_id}")
            print(f"    Format: {template.template_format}")
            print(f"    Size: {len(template.template_content)} characters")
            
            # Show first few lines of template
            lines = template.template_content.split('\n')[:5]
            print(f"    Preview:")
            for line in lines:
                print(f"      {line}")
            if len(template.template_content.split('\n')) > 5:
                print(f"      ... ({len(template.template_content.split('\n')) - 5} more lines)")
    
except Exception as e:
    print(f"ERROR: Infrastructure generation failed: {e}")

In [ ]:
# Example: Environment promotion workflow
try:
    # Deploy to development first
    dev_deployment = deployment_manager.create_deployment(
        name="customer-io-pipelines",
        version="2.2.0",
        environment=Environment.DEVELOPMENT,
        created_by="ci-cd-system"
    )
    
    dev_result = deployment_manager.deploy_application(dev_deployment)
    print(f"Development Deployment: {dev_result.status}")
    
    if dev_result.is_successful():
        # Promote to staging
        staging_result = deployment_manager.promote_deployment(
            from_env=Environment.DEVELOPMENT,
            to_env=Environment.STAGING,
            version="2.2.0",
            created_by="release-manager"
        )
        
        print(f"Staging Promotion: {staging_result.status}")
        
        if staging_result.is_successful():
            # Promote to production
            prod_result = deployment_manager.promote_deployment(
                from_env=Environment.STAGING,
                to_env=Environment.PRODUCTION,
                version="2.2.0",
                created_by="release-manager"
            )
            
            print(f"Production Promotion: {prod_result.status}")
            print(f"Production Strategy Used: {prod_result.deployment_id}")
        else:
            print("Staging deployment failed, skipping production promotion")
    else:
        print("Development deployment failed, skipping promotions")
    
except Exception as e:
    print(f"ERROR: Promotion workflow failed: {e}")

In [ ]:
# Example: Create configuration bundle
try:
    import tempfile
    
    # Create temporary directory for the bundle
    with tempfile.TemporaryDirectory() as temp_dir:
        bundle_files = deployment_manager.create_environment_config_bundle(
            environment=Environment.STAGING,
            output_dir=temp_dir
        )
        
        print("Configuration Bundle Created:")
        for bundle_type, file_path in bundle_files.items():
            file_size = os.path.getsize(file_path)
            print(f"  {bundle_type}: {os.path.basename(file_path)} ({file_size} bytes)")
            
            # Show content preview for smaller files
            if file_size < 1000 and bundle_type == "config":
                with open(file_path, 'r') as f:
                    content = f.read()
                    lines = content.split('\n')[:10]
                    print(f"    Preview:")
                    for line in lines:
                        print(f"      {line}")
                    if len(content.split('\n')) > 10:
                        print(f"      ... (truncated)")
        
        print(f"\nBundle created in: {temp_dir}")
        print("Note: Files are in a temporary directory and will be cleaned up automatically")
    
except Exception as e:
    print(f"ERROR: Bundle creation failed: {e}")

In [ ]:
# Example: Deployment dashboard and monitoring
try:
    # Get comprehensive dashboard data
    dashboard = deployment_manager.get_deployment_dashboard()
    
    print("Deployment Dashboard:")
    print(f"  System Uptime: {dashboard['system']['uptime_seconds']:.1f} seconds")
    
    print(f"\n  Deployment Statistics:")
    deployments = dashboard['deployments']
    print(f"    Total Deployments: {deployments['total_deployments']}")
    print(f"    Success Rate: {deployments['success_rate_percent']:.1f}%")
    print(f"    Average Duration: {deployments['average_deployment_time_seconds']:.1f}s")
    print(f"    Rollbacks Performed: {deployments['rollbacks_performed']}")
    print(f"    Active Deployments: {deployments['active_deployments']}")
    
    print(f"\n  Environment Status:")
    for env, stats in deployments['by_environment'].items():
        print(f"    {env}:")
        print(f"      Total: {stats['total']}")
        print(f"      Successful: {stats['successful']}")
        if stats['last_deployment']:
            print(f"      Last Deployment: {stats['last_deployment']}")
        else:
            print(f"      Last Deployment: Never")
    
    print(f"\n  Configuration:")
    config = dashboard['configuration']
    print(f"    Environments: {config['environments_registered']}")
    print(f"    Total Secrets: {config['total_secrets']}")
    print(f"    Encryption Enabled: {config['encryption_enabled']}")
    
    print(f"\n  Infrastructure:")
    infra = dashboard['infrastructure']
    print(f"    Templates Generated: {infra['templates_generated']}")
    print(f"    Supported Providers: {', '.join(infra['supported_providers'])}")
    
    # Get metrics summary
    metrics = deployment_manager.get_metrics()
    print(f"\nDeployment Manager Features:")
    for feature, enabled in metrics['features'].items():
        status = "ENABLED" if enabled else "DISABLED"
        print(f"  [{status}] {feature.replace('_', ' ').title()}")
    
    print("\nSUCCESS: Comprehensive production deployment system operational")
    
except Exception as e:
    print(f"ERROR: Dashboard retrieval failed: {e}")

## Summary

This notebook demonstrates a comprehensive production deployment solution for Customer.IO data pipelines including:

**Core Features:**
- **ConfigurationManager**: Secure configuration and secrets management with encryption
- **InfrastructureGenerator**: Infrastructure as Code (IaC) templates for multiple providers
- **DeploymentEngine**: Core deployment engine with multiple strategies and rollback capabilities
- **DeploymentManager**: Unified management of all deployment components

**Key Capabilities:**
- Multi-environment support (development, staging, production)
- Multiple deployment strategies (rolling update, blue-green, canary, recreate)
- Infrastructure as Code generation (Kubernetes, Docker Compose, Terraform)
- Secure secrets management with encryption
- Automated health checks and rollback capabilities
- Environment promotion workflows
- Security hardening with configurable levels
- Configuration bundle generation
- Comprehensive monitoring and dashboard

**Infrastructure Support:**
- Kubernetes deployments with HPA and security contexts
- Docker Compose with health checks and monitoring
- AWS ECS with Terraform templates
- Multi-cloud provider support
- Container orchestration and scaling

**Security & Compliance:**
- Secret encryption and secure storage
- Security context configuration
- Compliance with security levels
- Audit trail and deployment history
- Role-based deployment tracking

The system is designed for enterprise production use with comprehensive error handling, rollback capabilities, and support for complex deployment workflows. Ready for integration with CI/CD pipelines and infrastructure automation tools.